# Scenario 15

# Anlisis detallado escenario 15

In [1]:
import json
import os
import math
import copy # Para crear copias de los diccionarios si es necesario

# --- Constantes ---
CTI = 0.2  # Intervalo de tiempo de coordinación típico (en segundos)
TARGET_SCENARIO_ID = "scenario_15" # Escenario específico a analizar

# --- Ruta del archivo ---
input_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_optimization.json"

print(f"Archivo de entrada: {input_file}")
print(f"Analizando SOLAMENTE para: '{TARGET_SCENARIO_ID}'")

# Listas para clasificar los pares del escenario objetivo
coordinated_pairs = []
uncoordinated_pairs = []

try:
    # 1. Cargar el archivo JSON (lista de pares)
    print("Cargando datos (lista de pares)...")
    with open(input_file, 'r') as f:
        relay_pairs_data = json.load(f)
    print("Datos cargados correctamente.")

    # Validar que es una lista
    if not isinstance(relay_pairs_data, list):
        raise TypeError(f"Error: El archivo {input_file} no contiene una lista JSON.")

    # 2. Procesar cada par de relés, filtrando por scenario_id
    print(f"Calculando delta_t, mt y clasificando pares para '{TARGET_SCENARIO_ID}'...")
    total_pairs_read = 0
    scenario_pairs_found = 0
    skipped_pairs_count = 0 # Pares omitidos DENTRO del scenario_id objetivo

    for pair_entry in relay_pairs_data:
        total_pairs_read += 1
        if not isinstance(pair_entry, dict):
            # Contar como leído pero no procesado si no es diccionario
            continue

        # --- FILTRAR POR SCENARIO_ID ---
        current_scenario_id = pair_entry.get("scenario_id")
        if current_scenario_id != TARGET_SCENARIO_ID:
            continue # Omitir este par si no es del escenario objetivo
        # ---------------------------------

        # Si llegamos aquí, el par pertenece a TARGET_SCENARIO_ID
        scenario_pairs_found += 1

        # Obtener información de los relés de forma segura
        main_relay_info = pair_entry.get('main_relay')
        backup_relay_info = pair_entry.get('backup_relay')

        if not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            print(f"Advertencia ({TARGET_SCENARIO_ID}): Falta información de relé principal o de respaldo en: {pair_entry.get('line', 'N/A')}-{pair_entry.get('fault', 'N/A')}. Par omitido.")
            skipped_pairs_count += 1
            continue

        # Obtener tiempos de operación de forma segura
        main_time = main_relay_info.get('Time_out')
        backup_time = backup_relay_info.get('Time_out')

        # Validar que los tiempos son números
        if not isinstance(main_time, (int, float)) or not isinstance(backup_time, (int, float)):
            print(f"Advertencia ({TARGET_SCENARIO_ID}): Tiempo(s) de operación no numéricos o faltantes en: {pair_entry.get('line', 'N/A')}-{pair_entry.get('fault', 'N/A')} (Main: {main_time}, Backup: {backup_time}). Par omitido.")
            skipped_pairs_count += 1
            continue

        # --- Realizar Cálculos ---
        delta_t = backup_time - main_time - CTI
        mt = (delta_t - abs(delta_t)) / 2  # Penalización solo si delta_t es negativo

        # Crear una copia del par y añadirle los resultados del cálculo
        pair_info = copy.deepcopy(pair_entry)
        pair_info['delta_t'] = delta_t
        pair_info['mt'] = mt

        # --- Clasificar (solo pares del TARGET_SCENARIO_ID) ---
        if delta_t >= 0:
            coordinated_pairs.append(pair_info)
        else:
            uncoordinated_pairs.append(pair_info)

    print("Procesamiento de pares completado.")

    # 3. Calcular Métricas Finales (solo para TARGET_SCENARIO_ID)
    if scenario_pairs_found == 0:
         print (f"No se encontraron pares válidos para '{TARGET_SCENARIO_ID}' en el archivo.")
    else:
        total_valid_pairs_scenario = len(coordinated_pairs) + len(uncoordinated_pairs) # Pares válidos DENTRO del escenario
        miscoordination_count_scenario = len(uncoordinated_pairs)
        # Sumar 'mt' solo de los pares del escenario objetivo (que son los únicos en las listas)
        tmt_total_scenario = sum(pair["mt"] for pair in coordinated_pairs + uncoordinated_pairs)


        # 4. Imprimir Resultados
        print(f"\n--- Resultados del Análisis de Coordinación para '{TARGET_SCENARIO_ID}' ---")
        print(f"Total de pares leídos del archivo: {total_pairs_read}")
        print(f"Total de pares encontrados para '{TARGET_SCENARIO_ID}': {scenario_pairs_found}")
        if skipped_pairs_count > 0:
            print(f"Pares omitidos DENTRO de '{TARGET_SCENARIO_ID}' (datos inválidos/faltantes): {skipped_pairs_count}")
        print(f"Total de pares válidos analizados para '{TARGET_SCENARIO_ID}': {total_valid_pairs_scenario}")
        print(f"Número de pares coordinados (delta_t >= 0) en '{TARGET_SCENARIO_ID}': {len(coordinated_pairs)}")
        print(f"Número de pares DESCOORDINADOS (delta_t < 0) en '{TARGET_SCENARIO_ID}': {miscoordination_count_scenario}")
        print(f"Suma total de penalización por descoordinación (TMT Total) para '{TARGET_SCENARIO_ID}': {tmt_total_scenario:.5f}") # Imprimir con 5 decimales

except FileNotFoundError:
    print(f"Error CRÍTICO: No se pudo encontrar el archivo de entrada: {input_file}")
except TypeError as e:
    print(f"Error CRÍTICO: Problema con el tipo de datos esperado en el archivo JSON: {e}")
except json.JSONDecodeError as e:
    print(f"Error CRÍTICO: El archivo de entrada JSON ({input_file}) está mal formado: {e}")
except Exception as e:
    import traceback
    print(f"Error inesperado durante el procesamiento: {e}")
    print("--- Traceback ---")
    traceback.print_exc()
    print("-----------------")
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np

# Preparar los datos para la gráfica
def prepare_data_for_plot(coordinated_pairs, uncoordinated_pairs):
    # Combinar ambas listas
    all_pairs = coordinated_pairs + uncoordinated_pairs
    
    # Crear un DataFrame para facilitar el manejo de datos
    pairs_data = []
    for idx, pair in enumerate(all_pairs):
        main_relay_name = pair.get('main_relay', {}).get('RelayID', f"Main_{idx}")
        backup_relay_name = pair.get('backup_relay', {}).get('RelayID', f"Backup_{idx}")
        pair_name = f"{main_relay_name}-{backup_relay_name}"
        
        pairs_data.append({
            'index': idx + 1,  # Para enumerar los pares
            'pair_name': pair_name,
            'main_relay': main_relay_name,
            'backup_relay': backup_relay_name,
            'mt': pair.get('mt', 0),
            'delta_t': pair.get('delta_t', 0),
            'status': 'Coordinado' if pair.get('delta_t', 0) >= 0 else 'Descoordinado',
            'line': pair.get('line', 'N/A'),
            'fault': pair.get('fault', 'N/A')
        })
    
    return pd.DataFrame(pairs_data)

# Crear la visualización
def plot_mt_values(df):
    # Ordenar por valor de MT (de peor a mejor)
    df_sorted = df.sort_values(by='mt')
    
    # Crear un gráfico de barras para MT
    fig = go.Figure()
    
    # Añadir barras para pares coordinados (en verde)
    coordinated_df = df_sorted[df_sorted['status'] == 'Coordinado']
    fig.add_trace(go.Bar(
        x=coordinated_df['index'],
        y=coordinated_df['mt'],
        name='Pares Coordinados',
        marker_color='green',
        text=coordinated_df['pair_name'],
        hovertemplate='<b>%{text}</b><br>MT: %{y:.5f}<br>Índice: %{x}<extra></extra>'
    ))
    
    # Añadir barras para pares descoordinados (en rojo)
    uncoordinated_df = df_sorted[df_sorted['status'] == 'Descoordinado']
    fig.add_trace(go.Bar(
        x=uncoordinated_df['index'],
        y=uncoordinated_df['mt'],
        name='Pares Descoordinados',
        marker_color='red',
        text=uncoordinated_df['pair_name'],
        hovertemplate='<b>%{text}</b><br>MT: %{y:.5f}<br>Índice: %{x}<extra></extra>'
    ))
    
    # Añadir una línea horizontal en y=0
    fig.add_shape(
        type='line',
        x0=0,
        y0=0,
        x1=len(df),
        y1=0,
        line=dict(color='black', width=1, dash='dash')
    )
    
    # Personalizar la apariencia
    fig.update_layout(
        title=f'Índice MT para {len(df)} Pares de Relés en el Escenario Base',
        xaxis_title='Índice del Par de Relés',
        yaxis_title='Valor MT (Penalización)',
        barmode='group',
        xaxis=dict(
            tickmode='linear',
            tick0=1,
            dtick=5,
        ),
        legend=dict(
            x=0.01,
            y=0.99,
            bgcolor='rgba(255, 255, 255, 0.5)',
            bordercolor='rgba(0, 0, 0, 0.5)'
        )
    )
    
    # Mostrar el gráfico
    fig.show()

    # También crear una gráfica de delta_t para ver la diferencia de tiempos
    fig_delta = px.scatter(
        df_sorted,
        x='index',
        y='delta_t',
        color='status',
        color_discrete_map={'Coordinado': 'green', 'Descoordinado': 'red'},
        hover_data=['pair_name', 'delta_t', 'mt'],
        title=f'Delta_t para {len(df)} Pares de Relés (CTI = {CTI}s)'
    )
    
    # Añadir línea de CTI
    fig_delta.add_shape(
        type='line',
        x0=0,
        y0=0,
        x1=len(df),
        y1=0,
        line=dict(color='black', width=1, dash='dash')
    )
    
    fig_delta.update_layout(
        xaxis_title='Índice del Par de Relés',
        yaxis_title='Delta_t - CTI (s)',
        xaxis=dict(
            tickmode='linear',
            tick0=1,
            dtick=5,
        )
    )
    
    fig_delta.show()
    
    # Gráfico de distribución de valores MT
    fig_hist = px.histogram(
        df_sorted, 
        x='mt',
        color='status',
        color_discrete_map={'Coordinado': 'green', 'Descoordinado': 'red'},
        marginal='box',
        title="Distribución de Valores MT"
    )
    fig_hist.update_layout(
        xaxis_title='Valor MT',
        yaxis_title='Frecuencia',
    )
    fig_hist.show()

# Ejecutar la visualización
try:
    # Preparar los datos
    df = prepare_data_for_plot(coordinated_pairs, uncoordinated_pairs)
    
    # Generar las gráficas
    plot_mt_values(df)
    
    # Mostrar un resumen estadístico
    print("\n--- Resumen Estadístico de MT ---")
    print(f"Mínimo: {df['mt'].min():.5f}")
    print(f"Máximo: {df['mt'].min():.5f}")
    print(f"Promedio: {df['mt'].mean():.5f}")
    print(f"Mediana: {df['mt'].median():.5f}")
    
    # Identificar los 5 pares con peor descoordinación (MT más negativo)
    if len(df[df['status'] == 'Descoordinado']) > 0:
        print("\n--- 5 Pares con Mayor Descoordinación ---")
        worst_pairs = df.sort_values(by='mt').head(5)
        for idx, row in worst_pairs.iterrows():
            print(f"Par: {row['pair_name']}, Línea: {row['line']}, Falla: {row['fault']}, MT: {row['mt']:.5f}, Delta_t: {row['delta_t']:.5f}")
    
except Exception as e:
    print(f"Error al crear las gráficas: {e}")


Archivo de entrada: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_optimization.json
Analizando SOLAMENTE para: 'scenario_15'
Cargando datos (lista de pares)...
Datos cargados correctamente.
Calculando delta_t, mt y clasificando pares para 'scenario_15'...
Procesamiento de pares completado.

--- Resultados del Análisis de Coordinación para 'scenario_15' ---
Total de pares leídos del archivo: 6800
Total de pares encontrados para 'scenario_15': 100
Total de pares válidos analizados para 'scenario_15': 100
Número de pares coordinados (delta_t >= 0) en 'scenario_15': 89
Número de pares DESCOORDINADOS (delta_t < 0) en 'scenario_15': 11
Suma total de penalización por descoordinación (TMT Total) para 'scenario_15': -20.22100



--- Resumen Estadístico de MT ---
Mínimo: -2.30160
Máximo: -2.30160
Promedio: -0.20221
Mediana: 0.00000

--- 5 Pares con Mayor Descoordinación ---
Par: Main_95-Backup_95, Línea: N/A, Falla: 90, MT: -2.30160, Delta_t: -2.30160
Par: Main_94-Backup_94, Línea: N/A, Falla: 90, MT: -2.30160, Delta_t: -2.30160
Par: Main_97-Backup_97, Línea: N/A, Falla: 10, MT: -2.27900, Delta_t: -2.27900
Par: Main_96-Backup_96, Línea: N/A, Falla: 10, MT: -2.27900, Delta_t: -2.27900
Par: Main_90-Backup_90, Línea: N/A, Falla: 10, MT: -2.10740, Delta_t: -2.10740


In [2]:
import json
import os
import math
import copy # Para crear copias de los diccionarios si es necesario

# --- Constantes ---
CTI = 0.2  # Intervalo de tiempo de coordinación típico (en segundos)
TARGET_SCENARIO_ID = "scenario_15" # Escenario específico a analizar

# --- Ruta del archivo ---
input_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_optimization.json"

print(f"Archivo de entrada: {input_file}")
print(f"Analizando SOLAMENTE para: '{TARGET_SCENARIO_ID}'")

# Listas para clasificar los pares del escenario objetivo
coordinated_pairs = []
uncoordinated_pairs = []

try:
    # 1. Cargar el archivo JSON (lista de pares)
    print("Cargando datos (lista de pares)...")
    with open(input_file, 'r') as f:
        relay_pairs_data = json.load(f)
    print("Datos cargados correctamente.")

    # Validar que es una lista
    if not isinstance(relay_pairs_data, list):
        raise TypeError(f"Error: El archivo {input_file} no contiene una lista JSON.")

    # 2. Procesar cada par de relés, filtrando por scenario_id
    print(f"Calculando delta_t, mt y clasificando pares para '{TARGET_SCENARIO_ID}'...")
    total_pairs_read = 0
    scenario_pairs_found = 0
    skipped_pairs_count = 0 # Pares omitidos DENTRO del scenario_id objetivo

    for pair_entry in relay_pairs_data:
        total_pairs_read += 1
        if not isinstance(pair_entry, dict):
            # Contar como leído pero no procesado si no es diccionario
            continue

        # --- FILTRAR POR SCENARIO_ID ---
        current_scenario_id = pair_entry.get("scenario_id")
        if current_scenario_id != TARGET_SCENARIO_ID:
            continue # Omitir este par si no es del escenario objetivo
        # ---------------------------------

        # Si llegamos aquí, el par pertenece a TARGET_SCENARIO_ID
        scenario_pairs_found += 1

        # Obtener información de los relés de forma segura
        main_relay_info = pair_entry.get('main_relay')
        backup_relay_info = pair_entry.get('backup_relay')

        if not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            print(f"Advertencia ({TARGET_SCENARIO_ID}): Falta información de relé principal o de respaldo en: {pair_entry.get('line', 'N/A')}-{pair_entry.get('fault', 'N/A')}. Par omitido.")
            skipped_pairs_count += 1
            continue

        # Obtener tiempos de operación de forma segura
        main_time = main_relay_info.get('Time_out')
        backup_time = backup_relay_info.get('Time_out')

        # Validar que los tiempos son números
        if not isinstance(main_time, (int, float)) or not isinstance(backup_time, (int, float)):
            print(f"Advertencia ({TARGET_SCENARIO_ID}): Tiempo(s) de operación no numéricos o faltantes en: {pair_entry.get('line', 'N/A')}-{pair_entry.get('fault', 'N/A')} (Main: {main_time}, Backup: {backup_time}). Par omitido.")
            skipped_pairs_count += 1
            continue

        # --- Realizar Cálculos ---
        delta_t = backup_time - main_time - CTI
        mt = (delta_t - abs(delta_t)) / 2  # Penalización solo si delta_t es negativo

        # Crear una copia del par y añadirle los resultados del cálculo
        pair_info = copy.deepcopy(pair_entry)
        pair_info['delta_t'] = delta_t
        pair_info['mt'] = mt

        # --- Clasificar (solo pares del TARGET_SCENARIO_ID) ---
        if delta_t >= 0:
            coordinated_pairs.append(pair_info)
        else:
            uncoordinated_pairs.append(pair_info)

    print("Procesamiento de pares completado.")

    # 3. Calcular Métricas Finales (solo para TARGET_SCENARIO_ID)
    if scenario_pairs_found == 0:
         print (f"No se encontraron pares válidos para '{TARGET_SCENARIO_ID}' en el archivo.")
    else:
        total_valid_pairs_scenario = len(coordinated_pairs) + len(uncoordinated_pairs) # Pares válidos DENTRO del escenario
        miscoordination_count_scenario = len(uncoordinated_pairs)
        # Sumar 'mt' solo de los pares del escenario objetivo (que son los únicos en las listas)
        tmt_total_scenario = sum(pair["mt"] for pair in coordinated_pairs + uncoordinated_pairs)


        # 4. Imprimir Resultados
        print(f"\n--- Resultados del Análisis de Coordinación para '{TARGET_SCENARIO_ID}' ---")
        print(f"Total de pares leídos del archivo: {total_pairs_read}")
        print(f"Total de pares encontrados para '{TARGET_SCENARIO_ID}': {scenario_pairs_found}")
        if skipped_pairs_count > 0:
            print(f"Pares omitidos DENTRO de '{TARGET_SCENARIO_ID}' (datos inválidos/faltantes): {skipped_pairs_count}")
        print(f"Total de pares válidos analizados para '{TARGET_SCENARIO_ID}': {total_valid_pairs_scenario}")
        print(f"Número de pares coordinados (delta_t >= 0) en '{TARGET_SCENARIO_ID}': {len(coordinated_pairs)}")
        print(f"Número de pares DESCOORDINADOS (delta_t < 0) en '{TARGET_SCENARIO_ID}': {miscoordination_count_scenario}")
        print(f"Suma total de penalización por descoordinación (TMT Total) para '{TARGET_SCENARIO_ID}': {tmt_total_scenario:.5f}") # Imprimir con 5 decimales

except FileNotFoundError:
    print(f"Error CRÍTICO: No se pudo encontrar el archivo de entrada: {input_file}")
except TypeError as e:
    print(f"Error CRÍTICO: Problema con el tipo de datos esperado en el archivo JSON: {e}")
except json.JSONDecodeError as e:
    print(f"Error CRÍTICO: El archivo de entrada JSON ({input_file}) está mal formado: {e}")
except Exception as e:
    import traceback
    print(f"Error inesperado durante el procesamiento: {e}")
    print("--- Traceback ---")
    traceback.print_exc()
    print("-----------------")

import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np

# Preparar los datos para la gráfica
def prepare_data_for_plot(coordinated_pairs, uncoordinated_pairs):
    # Combinar ambas listas
    all_pairs = coordinated_pairs + uncoordinated_pairs
    
    # Crear un DataFrame para facilitar el manejo de datos
    pairs_data = []
    for idx, pair in enumerate(all_pairs):
        # CORRECCIÓN: Cambiar 'RelayID' por 'relay' para obtener los identificadores correctos
        main_relay_name = pair.get('main_relay', {}).get('relay', f"Main_{idx}")
        backup_relay_name = pair.get('backup_relay', {}).get('relay', f"Backup_{idx}")
        pair_name = f"{main_relay_name}-{backup_relay_name}"
        
        # También obtener las líneas si están disponibles
        main_line = pair.get('main_relay', {}).get('line', 'N/A')
        backup_line = pair.get('backup_relay', {}).get('line', 'N/A')
        
        pairs_data.append({
            'index': idx + 1,  # Para enumerar los pares
            'pair_name': pair_name,
            'main_relay': main_relay_name,
            'backup_relay': backup_relay_name,
            'main_line': main_line,
            'backup_line': backup_line,
            'mt': pair.get('mt', 0),
            'delta_t': pair.get('delta_t', 0),
            'status': 'Coordinado' if pair.get('delta_t', 0) >= 0 else 'Descoordinado',
            'line': pair.get('line', 'N/A'),
            'fault': pair.get('fault', 'N/A')
        })
    
    return pd.DataFrame(pairs_data)

# Crear la visualización
def plot_mt_values(df):
    # Ordenar por valor de MT (de peor a mejor)
    df_sorted = df.sort_values(by='mt')
    
    # Crear un gráfico de barras para MT
    fig = go.Figure()
    
    # Añadir barras para pares coordinados (en verde)
    coordinated_df = df_sorted[df_sorted['status'] == 'Coordinado']
    fig.add_trace(go.Bar(
        x=coordinated_df['index'],
        y=coordinated_df['mt'],
        name='Pares Coordinados',
        marker_color='green',
        text=coordinated_df['pair_name'],
        hovertemplate='<b>%{text}</b><br>MT: %{y:.5f}<br>Índice: %{x}<extra></extra>'
    ))
    
    # Añadir barras para pares descoordinados (en rojo)
    uncoordinated_df = df_sorted[df_sorted['status'] == 'Descoordinado']
    fig.add_trace(go.Bar(
        x=uncoordinated_df['index'],
        y=uncoordinated_df['mt'],
        name='Pares Descoordinados',
        marker_color='red',
        text=uncoordinated_df['pair_name'],
        hovertemplate='<b>%{text}</b><br>MT: %{y:.5f}<br>Índice: %{x}<extra></extra>'
    ))
    
    # Añadir una línea horizontal en y=0
    fig.add_shape(
        type='line',
        x0=0,
        y0=0,
        x1=len(df),
        y1=0,
        line=dict(color='black', width=1, dash='dash')
    )
    
    # Personalizar la apariencia
    fig.update_layout(
        title=f'Índice MT para {len(df)} Pares de Relés en el Escenario Base',
        xaxis_title='Índice del Par de Relés',
        yaxis_title='Valor MT (Penalización)',
        barmode='group',
        xaxis=dict(
            tickmode='linear',
            tick0=1,
            dtick=5,
        ),
        legend=dict(
            x=0.01,
            y=0.99,
            bgcolor='rgba(255, 255, 255, 0.5)',
            bordercolor='rgba(0, 0, 0, 0.5)'
        )
    )
    
    # Mostrar el gráfico
    fig.show()

    # También crear una gráfica de delta_t para ver la diferencia de tiempos
    fig_delta = px.scatter(
        df_sorted,
        x='index',
        y='delta_t',
        color='status',
        color_discrete_map={'Coordinado': 'green', 'Descoordinado': 'red'},
        hover_data=['pair_name', 'delta_t', 'mt'],
        title=f'Delta_t para {len(df)} Pares de Relés (CTI = {CTI}s)'
    )
    
    # Añadir línea de CTI
    fig_delta.add_shape(
        type='line',
        x0=0,
        y0=0,
        x1=len(df),
        y1=0,
        line=dict(color='black', width=1, dash='dash')
    )
    
    fig_delta.update_layout(
        xaxis_title='Índice del Par de Relés',
        yaxis_title='Delta_t - CTI (s)',
        xaxis=dict(
            tickmode='linear',
            tick0=1,
            dtick=5,
        )
    )
    
    fig_delta.show()
    
    # Gráfico de distribución de valores MT
    fig_hist = px.histogram(
        df_sorted, 
        x='mt',
        color='status',
        color_discrete_map={'Coordinado': 'green', 'Descoordinado': 'red'},
        marginal='box',
        title="Distribución de Valores MT"
    )
    fig_hist.update_layout(
        xaxis_title='Valor MT',
        yaxis_title='Frecuencia',
    )
    fig_hist.show()

# Ejecutar la visualización
try:
    # Preparar los datos
    df = prepare_data_for_plot(coordinated_pairs, uncoordinated_pairs)
    
    # Generar las gráficas
    plot_mt_values(df)
    
    # Mostrar un resumen estadístico
    print("\n--- Resumen Estadístico de MT ---")
    print(f"Mínimo: {df['mt'].min():.5f}")
    print(f"Máximo: {df['mt'].max():.5f}")  # CORREGIDO: era min(), ahora es max()
    print(f"Promedio: {df['mt'].mean():.5f}")
    print(f"Mediana: {df['mt'].median():.5f}")
    
    # Identificar los 5 pares con peor descoordinación (MT más negativo)
    if len(df[df['status'] == 'Descoordinado']) > 0:
        print("\n--- 5 Pares con Mayor Descoordinación ---")
        worst_pairs = df.sort_values(by='mt').head(5)
        for idx, row in worst_pairs.iterrows():
            # MEJORADO: Incluir más información útil en el resumen
            print(f"Par: {row['pair_name']} (Líneas: {row['main_line']}-{row['backup_line']}), Falla: {row['fault']}, MT: {row['mt']:.5f}, Delta_t: {row['delta_t']:.5f}")
    
except Exception as e:
    print(f"Error al crear las gráficas: {e}")


Archivo de entrada: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_optimization.json
Analizando SOLAMENTE para: 'scenario_15'
Cargando datos (lista de pares)...
Datos cargados correctamente.
Calculando delta_t, mt y clasificando pares para 'scenario_15'...
Procesamiento de pares completado.

--- Resultados del Análisis de Coordinación para 'scenario_15' ---
Total de pares leídos del archivo: 6800
Total de pares encontrados para 'scenario_15': 100
Total de pares válidos analizados para 'scenario_15': 100
Número de pares coordinados (delta_t >= 0) en 'scenario_15': 89
Número de pares DESCOORDINADOS (delta_t < 0) en 'scenario_15': 11
Suma total de penalización por descoordinación (TMT Total) para 'scenario_15': -20.22100



--- Resumen Estadístico de MT ---
Mínimo: -2.30160
Máximo: 0.00000
Promedio: -0.20221
Mediana: 0.00000

--- 5 Pares con Mayor Descoordinación ---
Par: R45-R71 (Líneas: L8-9-L9-15), Falla: 90, MT: -2.30160, Delta_t: -2.30160
Par: R45-R46 (Líneas: L8-9-L9-10), Falla: 90, MT: -2.30160, Delta_t: -2.30160
Par: R9-R71 (Líneas: L9-10-L9-15), Falla: 10, MT: -2.27900, Delta_t: -2.27900
Par: R9-R8 (Líneas: L9-10-L8-9), Falla: 10, MT: -2.27900, Delta_t: -2.27900
Par: R2-R1 (Líneas: L2-3-L1-2), Falla: 10, MT: -2.10740, Delta_t: -2.10740


# 🚀🚀🚀

# Analisys

In [6]:
import json
import os
import copy
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# --- Constantes ---
CTI = 0.2  # Intervalo de tiempo de coordinación típico (en segundos)

# --- Ruta del archivo ---
input_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_optimization.json"

def analyze_all_scenarios():
    try:
        # 1. Cargar el archivo JSON
        print(f"Cargando datos desde: {input_file}")
        with open(input_file, 'r') as f:
            relay_pairs_data = json.load(f)
        
        if not isinstance(relay_pairs_data, list):
            raise TypeError(f"Error: El archivo {input_file} no contiene una lista JSON.")
        
        # Diccionarios para almacenar datos por escenario
        scenario_results = {}  # Estructura: {scenario_id: {'tmt': valor, 'coordinated': cantidad, 'uncoordinated': cantidad}}
        
        # 2. Procesar cada par y agrupar por escenario
        print("Calculando TMT por escenario...")
        total_pairs_read = 0
        skipped_pairs_count = 0
        
        for pair_entry in relay_pairs_data:
            total_pairs_read += 1
            if not isinstance(pair_entry, dict):
                skipped_pairs_count += 1
                continue
            
            scenario_id = pair_entry.get("scenario_id")
            if not scenario_id:
                skipped_pairs_count += 1
                continue
                
            # Inicializar datos del escenario si es la primera vez
            if scenario_id not in scenario_results:
                scenario_results[scenario_id] = {
                    'tmt': 0, 
                    'coordinated': 0, 
                    'uncoordinated': 0,
                    'total_valid': 0
                }
            
            # Obtener información de los relés
            main_relay_info = pair_entry.get('main_relay')
            backup_relay_info = pair_entry.get('backup_relay')
            
            if not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
                skipped_pairs_count += 1
                continue
                
            # Obtener tiempos de operación
            main_time = main_relay_info.get('Time_out')
            backup_time = backup_relay_info.get('Time_out')
            
            # Validar que los tiempos son números
            if not isinstance(main_time, (int, float)) or not isinstance(backup_time, (int, float)):
                skipped_pairs_count += 1
                continue
                
            # Cálculos
            delta_t = backup_time - main_time - CTI
            mt = (delta_t - abs(delta_t)) / 2  # Penalización solo si delta_t es negativo
            
            # Actualizar resultados del escenario
            scenario_results[scenario_id]['tmt'] += mt
            scenario_results[scenario_id]['total_valid'] += 1
            
            if delta_t >= 0:
                scenario_results[scenario_id]['coordinated'] += 1
            else:
                scenario_results[scenario_id]['uncoordinated'] += 1
        
        # 3. Preparar datos para graficar
        data_for_chart = []
        for scenario, data in scenario_results.items():
            data_for_chart.append({
                'Escenario': scenario,
                'TMT': data['tmt'],
                'Pares Coordinados': data['coordinated'],
                'Pares Descoordinados': data['uncoordinated'],
                'Total Pares Válidos': data['total_valid'],
                'Porcentaje Coordinación': 100 * data['coordinated'] / data['total_valid'] if data['total_valid'] > 0 else 0
            })
        
        # Convertir a DataFrame y ordenar por nombre de escenario
        df = pd.DataFrame(data_for_chart)
        df = df.sort_values(by='Escenario')
        
        # 4. Crear gráfico de barras con Plotly
        fig = px.bar(
            df, 
            x='Escenario', 
            y='TMT',
            text='TMT',
            color='TMT',
            color_continuous_scale='Viridis',
            title='Penalización Total por Descoordinación (TMT) por Escenario',
            labels={'TMT': 'Total Miscoordination Time'},
            hover_data=['Pares Coordinados', 'Pares Descoordinados', 'Total Pares Válidos', 'Porcentaje Coordinación']
        )
        
        # Personalizar el gráfico
        fig.update_traces(
            texttemplate='%{text:.5f}',
            textposition='outside'
        )
        
        fig.update_layout(
            xaxis_title='Escenarios',
            yaxis_title='TMT (Total Miscoordination Time)',
            xaxis={'categoryorder': 'array', 'categoryarray': df['Escenario'].tolist()},
            font=dict(size=12),
            height=600,
            width=1000
        )
        
        # 5. Crear un segundo gráfico: porcentaje de coordinación
        fig2 = px.bar(
            df,
            x='Escenario',
            y='Porcentaje Coordinación',
            text='Porcentaje Coordinación',
            color='Porcentaje Coordinación',
            color_continuous_scale='Turbo',
            title='Porcentaje de Pares Coordinados por Escenario',
            labels={'Porcentaje Coordinación': 'Porcentaje de Coordinación (%)'},
            hover_data=['Pares Coordinados', 'Pares Descoordinados', 'Total Pares Válidos', 'TMT']
        )
        
        fig2.update_traces(
            texttemplate='%{text:.1f}%',
            textposition='outside'
        )
        
        fig2.update_layout(
            xaxis_title='Escenarios',
            yaxis_title='Porcentaje de Coordinación (%)',
            xaxis={'categoryorder': 'array', 'categoryarray': df['Escenario'].tolist()},
            font=dict(size=12),
            height=600,
            width=1000
        )
        
        # 5. Mostrar tabla de resultados
        print("\n--- Resultados por Escenario ---")
        print(f"{'Escenario':<15} {'Pares Válidos':<15} {'Coordinados':<15} {'Descoordinados':<15} {'% Coordin.':<15} {'TMT':<15}")
        print("-" * 90)
        
        for index, row in df.iterrows():
            print(f"{row['Escenario']:<15} {int(row['Total Pares Válidos']):<15} {int(row['Pares Coordinados']):<15} {int(row['Pares Descoordinados']):<15} {row['Porcentaje Coordinación']:.1f}%{' ':<10} {row['TMT']:.5f}")
        
        print(f"\nTotal de pares leídos: {total_pairs_read}")
        print(f"Pares omitidos (datos inválidos/faltantes): {skipped_pairs_count}")
        
        # Guardar los gráficos antes de mostrarlos
        output_dir = os.path.dirname(input_file)
        
        # Guardar el primer gráfico (TMT)
        output_file_tmt = os.path.join(output_dir, "tmt_by_scenario.html")
        fig.write_html(output_file_tmt)
        print(f"\nGráfico de TMT guardado en: {output_file_tmt}")
        
        # Guardar el segundo gráfico (% Coordinación)
        output_file_coord = os.path.join(output_dir, "coordination_percentage_by_scenario.html")
        fig2.write_html(output_file_coord)
        print(f"Gráfico de Porcentaje de Coordinación guardado en: {output_file_coord}")
        
        # Mostrar los gráficos
        fig.show()
        fig2.show()
        
    except FileNotFoundError:
        print(f"Error CRÍTICO: No se pudo encontrar el archivo de entrada: {input_file}")
    except TypeError as e:
        print(f"Error CRÍTICO: Problema con el tipo de datos esperado en el archivo JSON: {e}")
    except json.JSONDecodeError as e:
        print(f"Error CRÍTICO: El archivo de entrada JSON ({input_file}) está mal formado: {e}")
    except Exception as e:
        import traceback
        print(f"Error inesperado durante el procesamiento: {e}")
        print("--- Traceback ---")
        traceback.print_exc()
        print("-----------------")
        
if __name__ == "__main__":
    analyze_all_scenarios()


Cargando datos desde: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_optimization.json
Calculando TMT por escenario...

--- Resultados por Escenario ---
Escenario       Pares Válidos   Coordinados     Descoordinados  % Coordin.      TMT            
------------------------------------------------------------------------------------------
scenario_1      100             87              13              87.0%           -0.73930
scenario_10     100             90              10              90.0%           -19.95510
scenario_11     100             91              9               91.0%           -17.32250
scenario_12     100             90              10              90.0%           -18.72680
scenario_13     100             88              12              88.0%           -19.71930
scenario_14     100             89              11              89.0%           -19.22260
scenario_15     100             89              11              8